In [2]:
#basics
import os 
import numpy as np
import pandas as pd


#preprocessing


#model bulding, validation and prediction
from tensorflow.keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation,\
        RepeatVector, TimeDistributed, ActivityRegularization
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional


Using TensorFlow backend.


#### File Reading

In [7]:
# Questions = []
# Answers = []
    
# with open(COMMENTS, 'r') as f:
#     comments = json.load(f)
    
# with open(TRAIN_DATA, 'r') as f:
#     reader = csv.reader(f, delimiter='|')
#     for row in reader:
#         ancestors = row[0].split(' ')
#         responses = row[1].split(' ')
#         Questions.append([comments[r]['text'] for r in ancestors])
#         Answers.append([comments[r]['text'] for r in responses])

# with open(TEST_DATA, 'r') as f:
#     reader = csv.reader(f, delimiter='|')
#     for row in reader:
#         ancestors = row[0].split(' ')
#         responses = row[1].split(' ')
#         Questions.append([comments[r]['text'] for r in ancestors])
#         Answers.append([comments[r]['text'] for r in responses])


# QnAdata = pd.DataFrame(np.column_stack((Questions[:500], Answers[:500])),columns = ["Questions","Answer_1", "Answer_2"])

# QnAdata.head()

# QnAdata['Questions'] = [' '.join(map(str, l)) for l in QnAdata['Questions']]

# QnAdata["QnAcomb"] = QnAdata["Questions"]+" "+QnAdata["Answer_1"]


# print(QnAdata.head())

In [3]:
 QnAdata = pd.read_csv('SARC_DS')

In [4]:
 QnAdata.head()

,Unnamed: 0,Questions,Answer_1,Answer_2,QnAcomb
0,0,I've been searching for the answer for this fo...,Religion must have the answer,It's obviously tracks from a giant water tract...,I've been searching for the answer for this fo...
1,1,"Michael Phelps Apologizes For ""Regrettable"" Be...",Wow...he smoked pot...oh lord hes such a horri...,"Wow, his girlfriend is uhm... Ah fuck it, he's...","Michael Phelps Apologizes For ""Regrettable"" Be..."
2,2,Utah wants to create a database to track the i...,I think the government should track every morm...,Another idea from the party that wants to get ...,Utah wants to create a database to track the i...
3,3,The Six Million Dead Jews of World War ONE!,"Oh right, *both* wars were just jewish conspir...","i know this seems strange but, what if he was ...",The Six Million Dead Jews of World War ONE! Oh...
4,4,WSJ begins the Jeb Bush campaign for 2016,Good luck with that.,time to get that shack in montana.,WSJ begins the Jeb Bush campaign for 2016 Good...


In [5]:
QnAdata = QnAdata.drop('Unnamed: 0', axis = 1)

In [6]:
QnAdata.head()

,Questions,Answer_1,Answer_2,QnAcomb
0,I've been searching for the answer for this fo...,Religion must have the answer,It's obviously tracks from a giant water tract...,I've been searching for the answer for this fo...
1,"Michael Phelps Apologizes For ""Regrettable"" Be...",Wow...he smoked pot...oh lord hes such a horri...,"Wow, his girlfriend is uhm... Ah fuck it, he's...","Michael Phelps Apologizes For ""Regrettable"" Be..."
2,Utah wants to create a database to track the i...,I think the government should track every morm...,Another idea from the party that wants to get ...,Utah wants to create a database to track the i...
3,The Six Million Dead Jews of World War ONE!,"Oh right, *both* wars were just jewish conspir...","i know this seems strange but, what if he was ...",The Six Million Dead Jews of World War ONE! Oh...
4,WSJ begins the Jeb Bush campaign for 2016,Good luck with that.,time to get that shack in montana.,WSJ begins the Jeb Bush campaign for 2016 Good...


#### Creating Vocabulary

In [7]:
import nltk
#nltk.download('punkt')
import collections

In [8]:
counter = collections.Counter()

for i in range(len(QnAdata)):
    for word in nltk.word_tokenize(QnAdata.iloc[i][3]):
        counter[word]+=1

word2idx = {w:(i+1) for i,(w,_) in enumerate(counter.most_common())}        
idx2word = {v:k for k,v in word2idx.items()}


idx2word[0] = "PAD"

vocab_size = len(word2idx)+1

print("\n\nVocabulary size:",vocab_size)



Vocabulary size: 144143


In [12]:
def encode(sentence, maxlen,vocab_size):
    indices = np.zeros((maxlen, vocab_size))
    for i, w in enumerate(nltk.word_tokenize(sentence)):
        if i == maxlen: break
        indices[i, word2idx[w]] = 1
    return indices

In [13]:
def decode(indices, calc_argmax=True):
    if calc_argmax:
        indices = np.argmax(indices, axis=-1)
    return ' '.join(idx2word[x] for x in indices)

The following code is used to vectorize the question and answers with the given maximum length for both questions and answers

In [ ]:
x = quesns_train[0]
decode(x)

In [78]:
x = quesns_train[0]
decode(x)

In [93]:
Questions = QnAdata['Questions'][:100]
Answers = QnAdata['Answer_1'][:100]

In [94]:
question_maxlen = 10
answer_maxlen = 20

In [95]:
##ORIGINAL
# def create_questions(question_maxlen,vocab_size):
#     question_idx = np.zeros(shape=(len(Questions),question_maxlen,vocab_size))
    
#     for q in range(len(Questions)):
#         question = encode(Questions[q],question_maxlen,vocab_size)

#         question_idx[i] = question 
#     return question_idx

# quesns_train = create_questions(question_maxlen=question_maxlen,vocab_size=vocab_size)

MemoryError: Unable to allocate 1.35 TiB for an array with shape (128541, 10, 144143) and data type float64

In [87]:
º###ERROR PORQUE ALGUNAS RESPUESTAS SON NÚMEROS

def create_questions(question_maxlen,vocab_size):
    question_idx = np.zeros(shape=(len(Questions),question_maxlen,vocab_size))
    
    for q in range(len(Questions)):
        if type(q) == str:
            question_idx[i] = encode(Questions[q],question_maxlen,vocab_size)
        else:
            pass
        
    return question_idx

quesns_train = create_questions(question_maxlen=question_maxlen,vocab_size=vocab_size)

def create_answers(answer_maxlen,vocab_size):
    answer_idx = np.zeros(shape=(len(Answers),answer_maxlen,vocab_size))
    
    for q in range(len(Answers)):
        if type(q) == str:
            answer_idx[i] = encode(Answers[q],answer_maxlen,vocab_size)
        else:
            pass

    return answer_idx

answs_train = create_answers(answer_maxlen=answer_maxlen,vocab_size=vocab_size)

#### Model architecture

In [47]:
from keras.layers import Input,Dense,Dropout,Activation
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers import RepeatVector,TimeDistributed,ActivityRegularization

In [48]:
n_hidden = 128

question_layer = Input(shape=(question_maxlen,vocab_size))

encoder_rnn = LSTM(n_hidden,dropout=0.2,recurrent_dropout=0.2)(question_layer)

repeat_encode = RepeatVector(answer_maxlen)(encoder_rnn)

dense_layer = TimeDistributed(Dense(vocab_size))(repeat_encode)

regularized_layer = ActivityRegularization(l2=1)(dense_layer)

softmax_layer = Activation('softmax')(regularized_layer)

model = Model([question_layer],[softmax_layer])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print (model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 144143)        0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               73867264  
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 20, 128)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 144143)        18594447  
_________________________________________________________________
activity_regularization_1 (A (None, 20, 144143)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 20, 144143)        0         
Total params: 92,461,711
Trainable params: 92,461,711
Non-trainable params: 0
_______________________________________________

In [ ]:
#Readers encouraged to try this
encoder_rnn = Bidirectional(LSTM(n_hidden,dropout=0.2,recurrent_dropout=0.2),merge_mode='concat')(question_layer)

##### Model Training

In [67]:
print(decode(quesns_train[0]))

PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD


In [49]:
quesns_train_2 = quesns_train.astype('float32')
answs_train_2 = answs_train.astype('float32')

In [50]:
model.fit(quesns_train_2, answs_train_2,batch_size=32,epochs=1,validation_split=0.05)

Train on 4417 samples, validate on 233 samples
Epoch 1/1
4417/4417 [==============================] - 534s 121ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [ ]:
#model.fit(quesns_train_2, answs_train_2,batch_size=32,epochs=30,validation_split=0.05)

#### Model prediciton

In [64]:
print(quesns_train_2[2])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [52]:
ans_pred = model.predict(quesns_train_2[0:3])

In [58]:
print (decode(ans_pred[0]))
print (decode(ans_pred[2]))

PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD


In [51]:
#REVISAR .fit_generator()